In [1]:
import spacy
import pandas as pd
from tqdm import tqdm
import pickle

## 初期化

In [2]:
keywords = set(["シャリ"])

def check_keyword(text : str):
    for keyword in keywords:
        if keyword in text:
            return True
    return False

In [3]:
stopwords = set([
    "シャリ",
    "お",
    "料理長",
    "　",
    "副",
    "いる",
    "する",
    "前",
    "くれる",
    "なる",
    "いう",
    "入れる",
    "ある",
    "同じ",
    "良い",
    "ない",
    "くる",
    "笑",
    "いい",
    "いく",
    "こと",
    "思う",
    "もの",
    "おる",
    "さん"
])

In [4]:
#例文（ChatGPTにより生成）
sample_text = "この度、特別な機会にある高級寿司店での食事を楽しむことができました。店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。メニューは多彩で、新鮮な魚介類の品揃えに感動しました。シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。口に入れるとほっと米がほころぶ。ここまでシャリが良い店は珍しい。価格は高めですが、その価値に見合う素晴らしい食体験でした。また訪れたいと思わせる高級寿司店でした。"

In [5]:
ginza = spacy.load("ja_ginza_electra")

c:\Users\brigh\anaconda3\envs\jpnlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
wanted_parts = set(["VERB", "ADJ", "NOUN"])

## データ読み込み

In [7]:
with open("reviews.csv") as f:
    reviews = f.read().split("\n")
    
len(reviews)

5068

## 「シャリ」からの近さでスコア化して集計
  
文ごとにスコアを与える  
「シャリ」を含む文を１として、その近隣の文も（シャリの話をしている可能性が高いので）ポワソン分布的にスコアを与える  
文内の単語はその文のスコアだけ加点される

In [8]:
#原文を文ごとにわけ、さらに形態素にわける
def breakdown(text):
    _text = text.replace("<br>", "\n")
    sents = ginza(_text).sents
    
    sents_text = []
    sents_tokenized = []
    for sent in sents:
        sents_text.append(str(sent))
        sents_tokenized.append([])
        for token in sent:
            sents_tokenized[-1].append(token)
            
    return sents_text, sents_tokenized

sample_sents, sample_tokens = breakdown(sample_text)

print(sample_sents, sample_tokens)

['この度、特別な機会にある高級寿司店での食事を楽しむことができました。', '店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。', 'メニューは多彩で、新鮮な魚介類の品揃えに感動しました。', 'シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。', '寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。', '特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。', 'もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。', '口に入れるとほっと米がほころぶ。', 'ここまでシャリが良い店は珍しい。', '価格は高めですが、その価値に見合う素晴らしい食体験でした。', 'また訪れたいと思わせる高級寿司店でした。'] [[この, 度, 、, 特別, な, 機会, に, ある, 高級, 寿司店, で, の, 食事, を, 楽しむ, こと, が, でき, まし, た, 。], [店内, は, 落ち着い, た, 雰囲気, で, 、, スタッフ, の, 対応, も, 素晴らしかっ, た, です, 。], [メニュー, は, 多彩, で, 、, 新鮮, な, 魚介類, の, 品揃え, に, 感動, し, まし, た, 。], [シャリ, は, 絶妙, な, 締め, 加減, で, 、, 口, に, 入れる, と, 心地よい, 食感, が, 広がり, ます, 。], [寿司ネタ, と, の, 相性, も, 抜群, で, 、, 一貫, 一貫, が, バランス, の, 取れ, た, 美味しさ, でし, た, 。], [特に, おすすめ, は, 、, トロ, や, ウニ, の, 握り寿司, で, 、, 口, の, 中, で, とろける, よう, な, 味わい, が, 堪能, でき, まし, た, 。], [もう, 一度, シャリ, の, 話, に, 戻り, ます, が, 、, シャリ, は, 、, 程よい, 酸味, と, 口, 溶け, の, 良, さ, が, 特徴, で, 、, 寿司ネタ, と, の, ハーモニー, が, 絶妙, でし, た, 。], [口, に, 入れる, と, ほっと, 米, が, ほころぶ, 。], [ここ, まで, シャリ,

In [9]:
#文章ごとのスコアを産出
def evaluate_sentent_score(sents):
    has_keyword = []
    sents_n = 0
    for sent in sents:
        has_keyword.append(check_keyword(str(sent)))
        sents_n += 1
        
    scores = [0.0] * sents_n
    
    def put_score(score, ind):
        if (ind < 0) or (len(scores) <= ind):
            return
        elif scores[ind] < score:
            scores[ind] = score
    
    #近くにあるものを加点
    for cnt in range(sents_n):
        if has_keyword[cnt]:
            put_score(0.2, cnt-1)
            put_score(1.0, cnt)
            put_score(0.3, cnt+1)
            put_score(0.1, cnt+2)
            put_score(0.01, cnt+3)
            
    #キーワード持ち文の間にあるものをブースト
    BETWEEN_MAX = 2
    for between in range(2, BETWEEN_MAX+1):
        for cnt in range(sents_n-between):
            if has_keyword[cnt] and has_keyword[cnt+between]:
                for cnt1 in range(cnt, cnt+between):
                    #キーワード持ちだと逆に減点されることになる
                    if not has_keyword[cnt1]:
                        put_score(0.8, cnt1)
    
    return scores

sample_scores = evaluate_sentent_score(sample_sents)

for cnt in range(len(sample_sents)):
    print(sample_scores[cnt], sample_sents[cnt])

0.0 この度、特別な機会にある高級寿司店での食事を楽しむことができました。
0.0 店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。
0.2 メニューは多彩で、新鮮な魚介類の品揃えに感動しました。
1.0 シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。
0.3 寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。
0.2 特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。
1.0 もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。
0.8 口に入れるとほっと米がほころぶ。
1.0 ここまでシャリが良い店は珍しい。
0.3 価格は高めですが、その価値に見合う素晴らしい食体験でした。
0.1 また訪れたいと思わせる高級寿司店でした。


In [10]:
scores = {}

def add_score(word, score):
    global scores
    
    if word in scores:
        scores[word] += score
    else:
        scores[word] = score

def set_score(token, score):
    if score < 0.001:
        #skip
        return
    
    add_score(token.lemma_, score)
    add_score(give_context(token), score)

def give_context(token):
    return token.lemma_ + "-" + token.head.lemma_

#単語にスコアを与える
def analyze(text):
    global scores
    
    if not check_keyword(text):
        return
    
    sents, tokens = breakdown(text)
    
    sents_scores = evaluate_sentent_score(sents)
    
    for cnt_sent in range(len(sents)):
        score = sents_scores[cnt_sent]
        used = set()
        for token in tokens[cnt_sent]:
            #品詞
            if (token.pos_ in wanted_parts) and (token.lemma_ not in stopwords) and (token.lemma_ not in used):
                set_score(token, score)
                used.add(token)

#デモ
analyze(sample_text)
print(scores)

#デモデータをリセット
scores = {}

{'メニュー': 0.2, 'メニュー-多彩': 0.2, '多彩': 0.2, '多彩-感動': 0.2, '新鮮': 0.2, '新鮮-魚介類': 0.2, '魚介類': 0.2, '魚介類-品揃え': 0.2, '品揃え': 0.2, '品揃え-感動': 0.2, '感動': 0.2, '感動-感動': 0.2, '絶妙': 2.0, '絶妙-加減': 1.0, '締め': 1.0, '締め-加減': 1.0, '加減': 1.0, '加減-広がる': 1.0, '口': 3.0, '口-入れる': 1.8, '心地よい': 1.0, '心地よい-食感': 1.0, '食感': 1.0, '食感-広がる': 1.0, '広がる': 1.0, '広がる-広がる': 1.0, '寿司ネタ': 1.3, '寿司ネタ-相性': 0.3, '相性': 0.3, '相性-抜群': 0.3, '抜群': 0.3, '抜群-美味しさ': 0.3, '一貫': 0.6, '一貫-一貫': 0.3, '一貫-美味しさ': 0.3, 'バランス': 0.3, 'バランス-取れる': 0.3, '取れる': 0.3, '取れる-美味しさ': 0.3, '美味しさ': 0.3, '美味しさ-美味しさ': 0.3, 'おすすめ': 0.2, 'おすすめ-堪能': 0.2, 'トロ': 0.2, 'トロ-ウニ': 0.2, 'ウニ': 0.2, 'ウニ-握り寿司': 0.2, '握り寿司': 0.2, '握り寿司-堪能': 0.2, '口-中': 0.2, '中': 0.2, '中-堪能': 0.2, 'とろける': 0.2, 'とろける-味わい': 0.2, '味わい': 0.2, '味わい-堪能': 0.2, '堪能': 0.2, '堪能-堪能': 0.2, '一度': 1.0, '一度-戻る': 1.0, '話': 1.0, '話-戻る': 1.0, '戻る': 1.0, '戻る-絶妙': 1.0, '程よい': 1.0, '程よい-酸味': 1.0, '酸味': 1.0, '酸味-良い': 1.0, '口-溶ける': 1.0, '溶ける': 1.0, '溶ける-良い': 1.0, '特徴': 1.0, '特徴-絶妙': 1.0, '寿司ネタ-ハーモニー': 1.0, 'ハーモニー'

In [13]:
#分析
scores = {}
for review in tqdm(reviews):
    analyze(review)


100%|██████████| 5068/5068 [1:24:25<00:00,  1.00it/s]  


### 結果

In [14]:
#結果を表示
# 辞書の値を基準にソートして、上位100件を表示
sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)

cnt = 0
for key, value in sorted_items:
    text = f"word: {key}  score: {value}"
    
    if cnt < 100:
        print(text)
        cnt += 1
    
    #保存
    with open("DistanceResult.txt", "a", encoding = "utf-8") as f:
        f.write(text + "\n")

word: ネタ  score: 851.8000000000009
word: 美味しい  score: 445.15000000000117
word: 赤酢  score: 365.53000000000026
word: 握り  score: 363.8199999999995
word: 寿司  score: 303.7699999999993
word: 食べる  score: 303.35
word: 鮨  score: 223.70999999999995
word: 美味しい-美味しい  score: 200.63999999999945
word: 口  score: 182.8099999999997
word: 赤  score: 179.81999999999985
word: 握る  score: 176.58999999999983
word: バランス  score: 173.75000000000003
word: 中  score: 166.99999999999963
word: 温度  score: 164.10999999999999
word: 味  score: 161.0499999999998
word: 酢  score: 158.24999999999994
word: 最高  score: 153.01999999999998
word: 　　  score: 149.39999999999995
word: 感  score: 149.32999999999993
word: 脂  score: 142.59999999999985
word: マグロ  score: 139.3
word: 赤酢-シャリ  score: 137.0
word: 白  score: 136.32999999999996
word: 合わせる  score: 132.67999999999995
word: 酸味  score: 131.94999999999996
word: 方  score: 129.58999999999992
word: ネタ-シャリ  score: 128.0
word: 合う  score: 126.47999999999995
word: 好み  score: 125.68999999999996

In [67]:
#保存
with open("DistanceData.pkl", "wb") as f:
    pickle.dump(scores, f)

## 